In [1]:
print("Hello")

Hello


In [2]:
%pwd

'c:\\Users\\malvi\\Desktop\\Generative AI\\Ent-to-End-Medical-Chatbot-Genai\\research'

In [3]:
import os

os.chdir("../")
%pwd

'c:\\Users\\malvi\\Desktop\\Generative AI\\Ent-to-End-Medical-Chatbot-Genai'

In [4]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [5]:
## extract the data from the pdf

def load_pdf_data(data):
    loader = DirectoryLoader(data, glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents



In [6]:
extracted_data = load_pdf_data(data= 'Data/')

In [7]:
# extracted_data

In [8]:
# perfrom the text splitting 

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [9]:
text_chunk = text_split(extracted_data)
print("length of text chunk", len(text_chunk))

length of text chunk 1304


In [10]:
# !pip install sentence-transformers


In [11]:
## download the embedding model from the huggingface

import sentence_transformers
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()


c:\Users\malvi\anaconda3\envs\ragmed\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\malvi\AppData\Local\Temp\ipykernel_19844\571915104.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
test_query = embeddings.embed_query("My name is Kuldeep Malviya")
print("Length of test query", len(test_query))

Length of test query 384


In [13]:
# test_query

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
# PINECONE_API_KEY

'pcsk_PACJT_NVQPHDfSvHbFawfzJ9AeHUm5zeFXJjKYpHB4AXcbU7T2UeGGrkAKMPHkRQYNi9E'

In [17]:
## We have to intialise the pinceone with the python code
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
import os


pc = Pinecone(api_key=PINECONE_API_KEY)
# Create a serverless index
index_name = "medibot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


In [29]:
# import os
# os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [31]:
# !pip -q install langchain-pinecone


In [18]:
from langchain.vectorstores import Pinecone
import pinecone

In [19]:
## to store the vector embeddings 
# embed each chunk and upsert the embeddings into pinecone index
from langchain.vectorstores import Pinecone
import pinecone


docsearch = Pinecone.from_documents(
    documents=text_chunk,
    index_name=index_name,
    embedding=embeddings,
)



In [20]:
## load the existing index from pinecone
from langchain.vectorstores import Pinecone
import pinecone

docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [21]:
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k": 3})

In [23]:
retrieved_docs = retriever.invoke("What is Genetic Disorders?")
retrieved_docs

[Document(metadata={'page': 8.0, 'source': 'Data\\Current Essentials of Medicine(1)(1).pdf'}, page_content='viii Contributors\nKirsten Neudoerffer Kangelaris, MD \nResearch Fellow, Division of Hospital Medicine, University of\nCalifornia, San Francisco, California\nSelected Genetic Disorders\nHelen Kao, MD \nAssistant Professor, Division of Geriatrics, Department of Medicine,\nUniversity of California, San Francisco, San Francisco, California \nGeriatrics\nKewchang Lee, MD \nAssociate Clinical Professor of Psychiatry, University of California,\nSan Francisco; Director of Psychiatry Consultation, San Francisco\nVeterans Affairs Medical Center, San Francisco, California\nPsychiatric Disorders\nJoan C. Lo, MD \nResearch Scientist, Division of Research, Kaiser Permanente\nNorthern California; Associate Clinical Professor of Medicine,\nUniversity of California, San Francisco, Oakland, California\nEndocrine Disorders\nMichael P. Lukela, MD\nDirector, Medicine-Pediatrics Residency Program, Un

In [24]:
## load the groq api key 
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [49]:
# GROQ_API_KEY

In [25]:
## now we will integrate the llm 


from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    max_tokens = 500,
    groq_api_key = GROQ_API_KEY,
    model_name = "llama-3.3-70b-specdec"
)

In [26]:
## create the complete chain for the llm

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import  ChatPromptTemplate



system_prompt = ("You are an expert medical assistant with comprehensive knowledge across all fields of medicine. "
                "Use the following pieces of retrieved context to answer."
                "the question if you don't know the answer say that you"
                "don't know use three sentences maximum and keep the"
                "answer concise."

                "\n\n"
                "{context}"
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [27]:
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)


In [28]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain) 

In [29]:
response = rag_chain.invoke({"input": "What is  Common Disorders of the Eye?"})
print(response['answer'])

Common Disorders of the Eye refers to a range of conditions that affect the eye, including age-related macular degeneration, blepharitis, cataract, conjunctivitis, and others. These conditions are listed in the provided index from pages 509-528. They include various eye problems such as glaucoma, retinal detachment, and uveitis.


In [30]:
response = rag_chain.invoke({"input": "What is  Machine learning?"})
print(response['answer'])

Machine learning is a subset of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. It enables computers to automatically improve their performance on a task based on experience, allowing for applications in image recognition, natural language processing, and more. I don't have specific medical context for this question, but it's a general concept applicable to various fields, including medicine.
